In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data').get('EdbName')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def main():
    # List of indicator IDs you want to fetch
    indicator_ids = ['C2507241525309415','C2507241522066415','C2503171216383686','C2507251526428683','C2506191042028672','s002954804',
                     'C2506271543318132','C2507251613442433','C2507251611186872','ID01006262','C2201191283','C2301185622','m0067416','s002954804']  # Add more IDs as needed

    # Dictionary to store DataFrames for each indicator
    data_frames = {}

    for indicator_id in indicator_ids:
        data = fetch_indicator_details(indicator_id)
        if data:
            # Create a DataFrame with DataTime as index
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            # Only keep the 'Value' column and rename it to the indicator ID
            df = df[['Value']].rename(columns={'Value': fetch_indicator_name(indicator_id)})
            data_frames[indicator_id] = df

    # Concatenate all DataFrames along the columns
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print(result_df.info())
        result_df.to_excel("data_input/山东丙烯主流价_SC指数拟合残差_原油指数.xlsx")
        print("Data saved successfully as '山东丙烯主流价_SC指数拟合残差_原油指数.xlsx'")



if __name__ == "__main__":
    main()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4503 entries, 2003-01-31 to 2025-10-24
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   山东丙烯主流价-SC指数拟合残差/结算价:原油指数     1894 non-null   float64
 1   丙烯/期末库存/中国（周）超季节性/3年          96 non-null     float64
 2   山东汽柴油加权裂解价差                   2125 non-null   float64
 3   丙烯上游-下游开工/不含PP/LZ同差           99 non-null     float64
 4   石脑油价格（CFR日本）-Brent原油期货价格      3614 non-null   float64
 5   LME铜价                         3612 non-null   float64
 6   PP-SC（期货指数）拟合残差/结算价:原油指数      1840 non-null   float64
 7   丙烯：现货外销量：华东地区（除山东）（周）超季节性/3年  84 non-null     float64
 8   丙烯现货外销量（除山东）超季节性/3年           84 non-null     float64
 9   秦皇岛动力煤价格Q5500                 3831 non-null   float64
 10  新增社融滚动12个月累计同比                273 non-null    float64
 11  LME铜价同比                       3359 non-null   float64
 12  螺纹期货价格                        4028 non-null 

Data saved successfully as '山东丙烯主流价_SC指数拟合残差_原油指数.xlsx'
